In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np

In [3]:
X = np.load('train_x.npy', allow_pickle=True)
y = np.load('train_y.npy', allow_pickle=True)

In [12]:
input_layer = layers.Input(shape=(200, 4))

conv1 = layers.Conv1D(filters=200, kernel_size=13, activation='relu', padding='same', name='conv1')(input_layer)
pool1 = layers.MaxPool1D(pool_size=6)(conv1)

positions = tf.range(pool1.shape[1])
context = layers.Embedding(input_dim=pool1.shape[1], output_dim=pool1.shape[2])(positions)

contextual_meaning = tf.add(pool1, context)
inputs = layers.Dense(200)(contextual_meaning)

query = layers.Dense(200)(inputs)
value = layers.Dense(200)(inputs)

attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32)(query, value, return_attention_scores=True)
attention_inputs = tf.add(inputs, attention)

fcl1 = layers.Dense(256, activation='relu')(attention_inputs)
fcl2 = layers.Dense(256, activation='relu')(fcl1)

flat = layers.Flatten()(fcl2)

output = layers.Dense(1, activation='sigmoid')(flat)

In [13]:
model = Model(inputs=input_layer, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, validation_split=0.1)

Epoch 1/10
282/282 [==============================] - 14s 47ms/step - loss: 0.6785 - accuracy: 0.5422 - val_loss: 0.1447 - val_accuracy: 0.9530
Epoch 2/10
282/282 [==============================] - 13s 48ms/step - loss: 0.1299 - accuracy: 0.9661 - val_loss: 0.1089 - val_accuracy: 0.9750
Epoch 3/10
282/282 [==============================] - 18s 64ms/step - loss: 0.1128 - accuracy: 0.9718 - val_loss: 0.1377 - val_accuracy: 0.9740
Epoch 4/10
282/282 [==============================] - 20s 70ms/step - loss: 0.1151 - accuracy: 0.9738 - val_loss: 0.1286 - val_accuracy: 0.9750
Epoch 5/10
282/282 [==============================] - 20s 70ms/step - loss: 0.1000 - accuracy: 0.9761 - val_loss: 0.1048 - val_accuracy: 0.9750
Epoch 6/10
282/282 [==============================] - 22s 76ms/step - loss: 0.1002 - accuracy: 0.9744 - val_loss: 0.1238 - val_accuracy: 0.9730
Epoch 7/10
282/282 [==============================] - 20s 71ms/step - loss: 0.1151 - accuracy: 0.9689 - val_loss: 0.1068 - val_accuracy: